In [6]:
import pandas as pd
import numpy as np
import torch
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

from autogluon.tabular import TabularPredictor

def check_gpu_status():
    """GPU 상태 확인"""
    print("🔍 GPU 상태 확인:")
    print(f"PyTorch CUDA 사용 가능: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"CUDA 버전: {torch.version.cuda}")
        print(f"GPU 개수: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            gpu_name = torch.cuda.get_device_name(i)
            memory = torch.cuda.get_device_properties(i).total_memory / 1024**3
            print(f"GPU {i}: {gpu_name} ({memory:.1f} GB)")
        return True
    else:
        print("❌ GPU를 사용할 수 없습니다.")
        return False

def get_fixed_gpu_hyperparameters():
    """수정된 GPU 최적화 하이퍼파라미터 (num_cpus 오류 해결)"""
    
    # CPU 코어 수 자동 감지
    cpu_count = os.cpu_count() or 2
    print(f"감지된 CPU 코어 수: {cpu_count}")
    
    gpu_hyperparameters = {
        # XGBoost with GPU (num_cpus 제거)
        'XGB': [
            {
                'n_estimators': 300,
                'learning_rate': 0.1,
                'max_depth': 6,
                'tree_method': 'gpu_hist',
                'gpu_id': 0,
                'objective': 'binary:logistic',
                'eval_metric': 'logloss'
            },
            {
                'n_estimators': 500,
                'learning_rate': 0.05,
                'max_depth': 8,
                'tree_method': 'gpu_hist',
                'gpu_id': 0,
                'objective': 'binary:logistic',
                'eval_metric': 'logloss'
            }
        ],
        
        # LightGBM with GPU (기본 하이퍼파라미터만 사용)
        'GBM': [
            {
                'num_boost_round': 300,
                'learning_rate': 0.1,
                'device': 'gpu',
                'objective': 'binary'
            },
            {
                'num_boost_round': 500,
                'learning_rate': 0.05,
                'device': 'gpu',
                'objective': 'binary'
            }
        ],
        
        # CatBoost with GPU
        'CAT': [
            {
                'iterations': 300,
                'learning_rate': 0.1,
                'depth': 6,
                'task_type': 'GPU',
                'devices': '0'
            },
            {
                'iterations': 500,
                'learning_rate': 0.05,
                'depth': 8,
                'task_type': 'GPU',
                'devices': '0'
            }
        ],
        
        # Neural Network with PyTorch (GPU 최적화)
        'NN_TORCH': [
            {
                'num_epochs': 100,
                'learning_rate': 0.01,
                'batch_size': 512,
                'activation': 'relu',
                'dropout_prob': 0.1
            },
            {
                'num_epochs': 200,
                'learning_rate': 0.005,
                'batch_size': 1024,
                'activation': 'relu',
                'dropout_prob': 0.2
            }
        ],
        
        # Random Forest (CPU, n_jobs 명시적 설정)
        'RF': [
            {
                'n_estimators': 200,
                'max_depth': 15,
                'n_jobs': cpu_count
            },
            {
                'n_estimators': 300,
                'max_depth': 20,
                'n_jobs': cpu_count
            }
        ],
        
        # Extra Trees (CPU, n_jobs 명시적 설정)
        'XT': [
            {
                'n_estimators': 200,
                'max_depth': 15,
                'n_jobs': cpu_count
            },
            {
                'n_estimators': 300,
                'max_depth': 20,
                'n_jobs': cpu_count
            }
        ]
    }
    
    return gpu_hyperparameters

def train_autogluon_gpu_fixed(X, y, test_size=0.2, time_limit=600, quality='medium_quality'):
    """수정된 GPU 최적화 AutoGluon 훈련"""
    print("🚀 수정된 GPU 최적화 AutoGluon 훈련 시작")
    
    # GPU 상태 확인
    gpu_available = check_gpu_status()
    
    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )
    
    print(f"\n📊 데이터 분할:")
    print(f"- 훈련: {X_train.shape[0]:,} 행, {X_train.shape[1]} 컬럼")
    print(f"- 테스트: {X_test.shape[0]:,} 행")
    
    # 훈련 데이터 준비
    train_data = X_train.copy()
    train_data['target'] = y_train
    
    # 문제 유형 결정
    n_classes = len(np.unique(y))
    problem_type = 'binary' if n_classes == 2 else 'multiclass'
    print(f"📊 문제 유형: {problem_type} ({n_classes}개 클래스)")
    
    # CPU 코어 수 명시적 설정 (auto 대신 구체적 숫자)
    cpu_count = os.cpu_count() or 2
    
    # 수정된 ag_args_fit (num_cpus를 구체적 숫자로)
    ag_args_fit = {
        'num_gpus': 1 if gpu_available else 0,
        'num_cpus': cpu_count,  # 'auto' 대신 구체적 숫자
        'auto_stack': True
    }
    
    print(f"🔧 AutoGluon 설정:")
    print(f"- GPU 사용: {ag_args_fit['num_gpus']}개")
    print(f"- CPU 사용: {ag_args_fit['num_cpus']}개")
    print(f"- 시간 제한: {time_limit//60}분")
    print(f"- 품질: {quality}")
    
    # 모델 경로 정리 (기존 모델이 있으면 삭제)
    model_path = './autogluon_gpu_models_fixed'
    if os.path.exists(model_path):
        import shutil
        shutil.rmtree(model_path)
        print(f"🗑️ 기존 모델 디렉토리 삭제: {model_path}")
    
    # 모델 생성
    predictor = TabularPredictor(
        label='target',
        problem_type=problem_type,
        eval_metric='accuracy',
        path=model_path,
        verbosity=2
    )
    
    # GPU 최적화 하이퍼파라미터 (수정된 버전)
    if gpu_available and quality in ['best_quality', 'high_quality']:
        hyperparameters = get_fixed_gpu_hyperparameters()
        print("🔥 GPU 최적화 하이퍼파라미터 사용")
    else:
        hyperparameters = 'default'
        print("💻 기본 하이퍼파라미터 사용")
    
    # 모델 훈련
    print(f"\n⏳ GPU 가속 훈련 시작...")
    start_time = pd.Timestamp.now()
    
    try:
        predictor.fit(
            train_data,
            time_limit=time_limit,
            presets=quality,
            hyperparameters=hyperparameters,
            ag_args_fit=ag_args_fit,
            holdout_frac=0.1,
            num_bag_folds=5,  # GPU에서 안정성을 위해 줄임
            num_bag_sets=1,   # 단순화
            num_stack_levels=1,  # 스태킹 레벨 줄임
            verbosity=2,
            dynamic_stacking=False  # 동적 스태킹 비활성화로 안정성 확보
        )
        
        end_time = pd.Timestamp.now()
        training_time = (end_time - start_time).total_seconds()
        print(f"✅ GPU 모드 훈련 완료! (소요 시간: {training_time/60:.1f}분)")
        
    except Exception as e:
        print(f"⚠️ GPU 모드 실패: {e}")
        print("🔄 CPU 모드로 재시도...")
        
        # 새로운 predictor 생성 (CPU용)
        cpu_model_path = './autogluon_cpu_models_fallback'
        if os.path.exists(cpu_model_path):
            import shutil
            shutil.rmtree(cpu_model_path)
        
        predictor = TabularPredictor(
            label='target',
            problem_type=problem_type,
            eval_metric='accuracy',
            path=cpu_model_path,
            verbosity=1
        )
        
        # CPU 전용 설정
        cpu_hyperparameters = {
            'GBM': {},
            'CAT': {},
            'XGB': {},
            'RF': {'n_jobs': cpu_count},
            'XT': {'n_jobs': cpu_count}
        }
        
        predictor.fit(
            train_data,
            time_limit=time_limit,
            presets='medium_quality',
            hyperparameters=cpu_hyperparameters,
            ag_args_fit={'num_gpus': 0, 'num_cpus': cpu_count},
            holdout_frac=0.1,
            num_bag_folds=3,
            num_stack_levels=1,
            verbosity=1,
            dynamic_stacking=False
        )
        
        end_time = pd.Timestamp.now()
        training_time = (end_time - start_time).total_seconds()
        print(f"✅ CPU 모드 훈련 완료! (소요 시간: {training_time/60:.1f}분)")
    
    # GPU 메모리 정리
    if gpu_available:
        torch.cuda.empty_cache()
    
    return predictor, X_test, y_test, training_time

def evaluate_model_performance(predictor, X_test, y_test, training_time):
    """모델 성능 평가"""
    print(f"\n📈 모델 성능 평가")
    
    try:
        # 예측 시간 측정
        start_time = pd.Timestamp.now()
        y_pred = predictor.predict(X_test)
        y_pred_proba = predictor.predict_proba(X_test)
        end_time = pd.Timestamp.now()
        
        prediction_time = (end_time - start_time).total_seconds()
        
        # 성능 지표
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f"🎯 성능 결과:")
        print(f"- 정확도: {accuracy:.4f}")
        print(f"- 훈련 시간: {training_time/60:.1f}분")
        print(f"- 예측 시간: {prediction_time:.2f}초")
        print(f"- 예측 속도: {len(X_test)/prediction_time:.0f} 샘플/초")
        
        # 모델 리더보드
        try:
            leaderboard = predictor.leaderboard(silent=True)
            print(f"\n🏆 상위 모델:")
            top_models = leaderboard.head(5)
            for idx, row in top_models.iterrows():
                model_name = row['model']
                score = row['score_val']
                print(f"  {model_name}: {score:.4f}")
        except Exception as e:
            print(f"리더보드 조회 실패: {e}")
            leaderboard = pd.DataFrame()
        
        return accuracy, y_pred, y_pred_proba, leaderboard
        
    except Exception as e:
        print(f"❗ 성능 평가 중 오류: {e}")
        return 0.0, None, None, pd.DataFrame()

def simple_autogluon_gpu(file_path, time_limit=6000, quality='medium_quality'):
    """간단한 AutoGluon GPU 파이프라인"""
    print("🌟 간단한 AutoGluon GPU 파이프라인")
    print("=" * 50)
    
    try:
        # 데이터 로드
        print("📁 데이터 로딩...")
        df = pd.read_csv(file_path)
        print(f"✅ 데이터 로드 완료: {df.shape}")
        
        # 전처리
        X = df.drop('result', axis=1)
        y = df['result']
        
        # 타겟 인코딩 (필요한 경우)
        if y.dtype == 'object' or y.dtype == 'bool':
            le = LabelEncoder()
            y = le.fit_transform(y)
        
        print(f"📊 전처리 완료: {X.shape}, 타겟 클래스 {len(np.unique(y))}개")
        
        # GPU 최적화 훈련
        predictor, X_test, y_test, training_time = train_autogluon_gpu_fixed(
            X, y, time_limit=time_limit, quality=quality
        )
        
        # 성능 평가
        accuracy, y_pred, y_pred_proba, leaderboard = evaluate_model_performance(
            predictor, X_test, y_test, training_time
        )
        
        print("\n" + "=" * 50)
        print(f"🎉 파이프라인 완료!")
        print(f"🎯 최종 정확도: {accuracy:.4f}")
        print(f"⏱️ 총 훈련 시간: {training_time/60:.1f}분")
        print("=" * 50)
        
        return predictor, accuracy, leaderboard
        
    except Exception as e:
        print(f"❗ 파이프라인 오류: {e}")
        print("💡 기본 CPU 모드를 시도해보세요.")
        return None, 0.0, pd.DataFrame()

def basic_cpu_fallback(file_path, time_limit=6000):
    """기본 CPU 폴백 모드"""
    print("💻 기본 CPU 폴백 모드")
    
    try:
        # 데이터 로드
        df = pd.read_csv(file_path)
        X = df.drop('result', axis=1)
        y = df['result']
        
        # 타겟 인코딩
        if y.dtype == 'object' or y.dtype == 'bool':
            le = LabelEncoder()
            y = le.fit_transform(y)
        
        # 데이터 분할
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # 훈련 데이터 준비
        train_data = X_train.copy()
        train_data['target'] = y_train
        
        # 간단한 CPU 모델
        predictor = TabularPredictor(
            label='target',
            problem_type='binary' if len(np.unique(y)) == 2 else 'multiclass',
            path='./autogluon_simple_cpu',
            verbosity=1
        )
        
        predictor.fit(
            train_data,
            time_limit=time_limit,
            presets='optimize_for_deployment',  # 빠른 훈련
            verbosity=1
        )
        
        # 예측 및 평가
        y_pred = predictor.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f"✅ CPU 폴백 완료! 정확도: {accuracy:.4f}")
        
        return predictor, accuracy
        
    except Exception as e:
        print(f"❗ CPU 폴백도 실패: {e}")
        return None, 0.0

# 사용 예시
if __name__ == "__main__":
    #file_path = '/home/elicer/ai/train/20230601.csv'
    file_path = '/home/elicer/ai/train/merged_output.csv'
    print("🎯 AutoGluon GPU 최적화 (수정 버전)")
    
    try:
        # GPU 최적화 버전 시도
        predictor, accuracy, leaderboard = simple_autogluon_gpu(
            file_path=file_path,
            time_limit=6000,  # 5분
            quality='medium_quality'
        )
        
        if predictor is not None:
            print(f"\n🚀 성공! 최종 정확도: {accuracy:.4f}")
        else:
            raise Exception("GPU 모드 실패")
            
    except Exception as e:
        print(f"\n⚠️ GPU 모드 실패: {e}")
        print("🔄 기본 CPU 모드로 전환...")
        
        # CPU 폴백
        predictor, accuracy = basic_cpu_fallback(file_path, time_limit=6000)
        
        if predictor is not None:
            print(f"💻 CPU 모드 성공! 정확도: {accuracy:.4f}")
        else:
            print("❌ 모든 모드 실패")

🎯 AutoGluon GPU 최적화 (수정 버전)
🌟 간단한 AutoGluon GPU 파이프라인
📁 데이터 로딩...
✅ 데이터 로드 완료: (2678129, 27)
📊 전처리 완료: (2678129, 26), 타겟 클래스 2개
🚀 수정된 GPU 최적화 AutoGluon 훈련 시작
🔍 GPU 상태 확인:
PyTorch CUDA 사용 가능: True
CUDA 버전: 12.4
GPU 개수: 1
GPU 0: NVIDIA A100 80GB PCIe MIG 1g.10gb (9.5 GB)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #107-Ubuntu SMP Wed Feb 7 13:26:48 UTC 2024
CPU Count:          2
Memory Avail:       19.27 GB / 24.00 GB (80.3%)
Disk Space Avail:   452.73 GB / 511.75 GB (88.5%)
Presets specified: ['medium_quality']



📊 데이터 분할:
- 훈련: 2,142,503 행, 26 컬럼
- 테스트: 535,626 행
📊 문제 유형: binary (2개 클래스)
🔧 AutoGluon 설정:
- GPU 사용: 1개
- CPU 사용: 2개
- 시간 제한: 100분
- 품질: medium_quality
🗑️ 기존 모델 디렉토리 삭제: ./autogluon_gpu_models_fixed
💻 기본 하이퍼파라미터 사용

⏳ GPU 가속 훈련 시작...


Beginning AutoGluon training ... Time limit = 6000s
AutoGluon will save models to "/home/elicer/ai/autogluon_gpu_models_fixed"
Train Data Rows:    2142503
Train Data Columns: 26
Label Column:       target
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    19734.99 MB
	Train Data (Original)  Memory Usage: 425.00 MB (2.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 5 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5

✅ GPU 모드 훈련 완료! (소요 시간: 100.4분)

📈 모델 성능 평가
🎯 성능 결과:
- 정확도: 0.9956
- 훈련 시간: 100.4분
- 예측 시간: 2197.78초
- 예측 속도: 244 샘플/초

🏆 상위 모델:
  LightGBM_BAG_L2: 0.9947
  WeightedEnsemble_L3: 0.9947
  LightGBMXT_BAG_L2: 0.9944
  LightGBMXT_BAG_L1: 0.9942
  WeightedEnsemble_L2: 0.9942

🎉 파이프라인 완료!
🎯 최종 정확도: 0.9956
⏱️ 총 훈련 시간: 100.4분

🚀 성공! 최종 정확도: 0.9956


In [9]:
import pandas as pd

def predict_unlabeled_file(predictor, test_file_path, output_file_path):
    """외부 테스트 파일에서 예측 결과 생성"""
    print("\n🔍 외부 테스트 파일 예측 시작")
    try:
        # 테스트 파일 로드
        test_data = pd.read_csv(test_file_path)
        print(f"✅ 테스트 파일 로드 완료: {test_data.shape}")

        # 예측 수행
        predictions = predictor.predict(test_data)

        # 결과 저장
        test_data['result'] = predictions
        test_data['result'] = test_data['result'].apply(lambda x: 'TRUE' if x == 1 else 'FALSE')
        test_data = test_data[['MMSI', 'result']]
        # 결과 파일 저장
        test_data.to_csv(output_file_path, index=False)
        print(f"✅ 결과 저장 완료: {output_file_path}")
        
    except Exception as e:
        print(f"❗ 테스트 파일 예측 중 오류: {e}")

# 사용 예시
if __name__ == "__main__":
    file_path = '/home/elicer/ai/train/20230601.csv'  # 훈련 데이터 파일 경로
    test_file_path = '/home/elicer/ai/20240701.csv'  # 테스트 파일 경로
    output_file_path = '/home/elicer/ai/test2_20240701.csv'  # 결과 저장 경로

    print("🎯 AutoGluon GPU 최적화 (수정 버전)")

    try:
        # # GPU 최적화 버전 시도
        # predictor, accuracy, leaderboard = simple_autogluon_gpu(
        #     file_path=file_path,
        #     time_limit=300,  # 5분
        #     quality='medium_quality'
        # )

        if predictor is not None:
            print(f"\n🚀 성공! 최종 정확도: {accuracy:.4f}")

            # 테스트 파일 예측
            predict_unlabeled_file(predictor, test_file_path, output_file_path)
        else:
            raise Exception("GPU 모드 실패")
            
    except Exception as e:
        print(f"\n⚠️ GPU 모드 실패: {e}")
        print("🔄 기본 CPU 모드로 전환...")

        # CPU 폴백
        predictor, accuracy = basic_cpu_fallback(file_path, time_limit=6000)

        if predictor is not None:
            print(f"💻 CPU 모드 성공! 정확도: {accuracy:.4f}")

            # 테스트 파일 예측
            predict_unlabeled_file(predictor, test_file_path, output_file_path)
        else:
            print("❌ 모든 모드 실패")

🎯 AutoGluon GPU 최적화 (수정 버전)

🚀 성공! 최종 정확도: 0.9956

🔍 외부 테스트 파일 예측 시작
✅ 테스트 파일 로드 완료: (6779, 26)
✅ 결과 저장 완료: /home/elicer/ai/test2_20240701.csv


In [ ]:
import pandas as pd
df = pd.read_csv('/home/elicer/ai/test_20240701.csv')
output_file_path = '/home/elicer/ai/test_20240701.csv'
df = df[['MMSI', 'result']]
df['result'] = df['result'].map({True: 'TRUE', False: 'FALSE'})
df.to_csv(output_file_path, index=False)